In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import network
import dataloader as dl

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
Tokenizer = dl.Tokenizer()

In [4]:
train_dataset = dl.SpamData('train.csv', Tokenizer, 180)
Tokenizer = train_dataset.get()
test_dataset = dl.SpamData('test.csv', Tokenizer, 180)
Tokenizer = test_dataset.get()

The word is already in the dictionary. Stupid monkey >:(
The word is already in the dictionary. Stupid monkey >:(
The word is already in the dictionary. Stupid monkey >:(


In [5]:
train_data = DataLoader(train_dataset, 64, True)
test_data = DataLoader(train_dataset, 64)

In [6]:
model = network.Network().to(device)

In [7]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        losses += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch completed. Avg loss: {losses/len(dataloader):>8f}")     

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [14]:
def using(model, data):
    model.eval()
    with torch.no_grad():
        data.to(device)
        pred = model(data)
        return Tokenizer.Idx2Word(pred.argmax().item()+1)

In [11]:
epochs = 50
for epoch in range(epochs):
    print(f"Epoch {epoch+1} is started!")
    train(train_data, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)

Epoch 1 is started!
Epoch completed. Avg loss: 9.528717
Test Error: 
 Accuracy: 84.9%, Avg loss: 6.697814
Epoch 2 is started!
Epoch completed. Avg loss: 6.247329
Test Error: 
 Accuracy: 83.6%, Avg loss: 5.539871
Epoch 3 is started!
Epoch completed. Avg loss: 5.129419
Test Error: 
 Accuracy: 85.2%, Avg loss: 4.748091
Epoch 4 is started!
Epoch completed. Avg loss: 4.754967
Test Error: 
 Accuracy: 84.7%, Avg loss: 4.337263
Epoch 5 is started!
Epoch completed. Avg loss: 4.240355
Test Error: 
 Accuracy: 85.1%, Avg loss: 4.009744
Epoch 6 is started!
Epoch completed. Avg loss: 4.289550
Test Error: 
 Accuracy: 85.8%, Avg loss: 3.883727
Epoch 7 is started!
Epoch completed. Avg loss: 3.757811
Test Error: 
 Accuracy: 85.3%, Avg loss: 3.585878
Epoch 8 is started!
Epoch completed. Avg loss: 3.672604
Test Error: 
 Accuracy: 84.1%, Avg loss: 3.485676
Epoch 9 is started!
Epoch completed. Avg loss: 3.428339
Test Error: 
 Accuracy: 85.7%, Avg loss: 3.302140
Epoch 10 is started!
Epoch completed. Avg loss

KeyboardInterrupt: 

In [16]:
import re
text = [0] * 180
pattern = r'[^a-zA-Z0-9\s]'
text_tmp = "Congratulations! You've won a $1,000 gift card. Click here to claim your reward. Act fast, this offer expires soon!"
text_tmp = re.sub(pattern, '', text_tmp)
text_tmp = text_tmp.split()
for i in range(len(text_tmp)):
    text[i] = Tokenizer.Word2Idx(text_tmp[i])
text = torch.tensor(text, dtype=torch.float)    
using(model, text)

'ham'